In [1]:
import pickle
import pandas as pd
import torch
pred_device = "cuda:0"
from transformers import AutoModelForCausalLM, AutoTokenizer
attrs = {}
zero = True
datasets = ["sst2","news","imdb"]
models = ["gemma-2b"]


for ds in datasets:
    if zero == True:
        for model in models:
            att_name = f"attrs/{ds}_gemma-2b-it_attr_quantized_False_zero.pcl"
            with open(att_name,"br") as f:
                attr = pickle.load(f)
                attr_df = pd.DataFrame(attr)
                attrs[f"{ds}_gemma-2b-it-zero"] = attr_df

            att_name = f"attrs/{ds}_predictor_{model}_{ds}_merged_attr_quantized_False.pcl"
            with open(att_name,"br") as f:
                attr = pickle.load(f)
                attr_df = pd.DataFrame(attr)
                attrs[f"{ds}_{model}-ft"] = attr_df
    else:
        for model in models:

            att_name = f"attrs/{ds}_{model}-it_attr_quantized_False.pcl"
            with open(att_name,"br") as f:
                attr = pickle.load(f)
                attr_df = pd.DataFrame(attr)
                attrs[f"{ds}_{model}-it"] = attr_df

            att_name = f"attrs/{ds}_predictor_{model}_{ds}_merged_attr_quantized_False.pcl"
            with open(att_name,"br") as f:
                attr = pickle.load(f)
                attr_df = pd.DataFrame(attr)
                attrs[f"{ds}_{model}-ft"] = attr_df


In [2]:
tokenizer = AutoTokenizer.from_pretrained("gemma-2b-it")

In [3]:
from collections import defaultdict
from tqdm import tqdm
from datasets import load_from_disk

import warnings
warnings.filterwarnings('ignore')
from transformers import logging
logging.set_verbosity(logging.ERROR)


In [5]:
import numpy as np
methods = ["unk","erase","mask","att-zero"]
methods = ["att-zero"]

using_fine_tuned = [False,True]
percentages = [0.1,0.2,0.3,0.4,0.5]
nums = [0,200]
datasets = ["sst2","news","imdb"]
pred_models = ["gemma-2b"]

for ds in datasets:

    torch.cuda.empty_cache()
    for pr_model in pred_models:
            
        for ft in using_fine_tuned:
            flipping_dict = defaultdict(list)
            for method in methods:
                

                torch.cuda.empty_cache()

                if ft:
                    pred_model_id = f"predictor_{pr_model}_{ds}_merged"
                    eval_data = attrs[f'{ds}_{pr_model}-ft']
                else:
                    
                    if zero:
                        pred_model_id = f"{pr_model}-it"
                        eval_data = attrs[f'{ds}_{pr_model}-it-zero']
                    else:
                        pred_model_id = f"{pr_model}-it"
                        eval_data = attrs[f'{ds}_{pr_model}-it']

                # if zero:
                #     pass
                # else:     
                pred_model = AutoModelForCausalLM.from_pretrained(pred_model_id,device_map=pred_device,
                        torch_dtype=torch.float16,trust_remote_code=True, attn_implementation="flash_attention_2")
                pred_tokenizer = AutoTokenizer.from_pretrained(pred_model_id)
                    # all_tokens = list(pred_tokenizer.get_vocab().keys())
                
                for i in tqdm(range(nums[0],nums[1])):

                    flipping_dict["sentence"].append(eval_data["comment"].iloc[i])
                    explanation_types = list(eval_data.columns[5:])
                    for exp in explanation_types:
                        flipped = False
                        attr = eval_data[exp].iloc[i]


                        for percent in percentages:
                            num_to_mask = int(np.ceil(percent * len(pred_tokenizer(eval_data.iloc[i].comment)["input_ids"])))

                            idx_to_mask = np.argsort(attr)[-int(num_to_mask):]

                            tokens = pred_tokenizer.convert_ids_to_tokens(pred_tokenizer(eval_data.prompt.iloc[i])["input_ids"])

                            if method == "erase":
                                for idss in idx_to_mask:
                                    tokens[idss] = ""
                            
                            elif method == "unk":
                                for idss in idx_to_mask:
                                    tokens[idss] = " <unk> "
                            elif method == "mask":
                                for idss in idx_to_mask:
                                    tokens[idss] = " <mask> "
                            
                                

                            sentence = pred_tokenizer.convert_tokens_to_string(tokens)
                            flipping_dict[f"{exp}_masked_sentence_{percent}_predictor_{pr_model}_ft_{ft}_{method}"].append(sentence)
                            
                            predicted_label = eval_data.predicted_id.iloc[i]
                            foil_id = eval_data.foil_id.iloc[i].tolist()
                            foil_sentiment = pred_tokenizer.convert_ids_to_tokens(foil_id)

                            


                            encoding = pred_tokenizer(sentence, return_tensors="pt")
                            
                            ids = encoding.input_ids.to(pred_device)
                            mask = encoding.attention_mask

                            if method == "att-zero":
                                mask[0][idx_to_mask] = 0
                            mask = mask.to(pred_device)

                            logits  = pred_model(input_ids = ids, attention_mask = mask).logits[0,-1]
                            if logits[foil_id] > logits[predicted_label]:
                                if flipped == False:
                                    flipped = True
                                    mask_percent = percent

                            if flipped == False and percent == 0.5:
                                mask_percent = percent
                        
                        flipping_dict[f"{exp}_mask_percent_predictor_{pr_model}_ft_{ft}_{method}"].append(mask_percent)
                        flipping_dict[f"{exp}_succuss_in_flipping_predictor_{pr_model}_ft_{ft}_{method}"].append(flipped)

            if zero:
                if ft:
                    with open(f"camel_mask2/{ds}_predictor_{pr_model}_ft_{ft}_{nums[0]}_{nums[1]}.pcl","+bw") as f:
                        pickle.dump(flipping_dict,f)
                else:

                    with open(f"camel_mask2/{ds}_predictor_{pr_model}_ft_{ft}_{nums[0]}_{nums[1]}_zero.pcl","+bw") as f:
                        pickle.dump(flipping_dict,f)
            else:
                with open(f"camel_mask2/{ds}_predictor_{pr_model}_ft_{ft}_{nums[0]}_{nums[1]}.pcl","+bw") as f:
                    pickle.dump(flipping_dict,f)
 


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 200/200 [01:22<00:00,  2.43it/s]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 200/200 [01:21<00:00,  2.44it/s]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

100%|██████████| 200/200 [01:39<00:00,  2.01it/s]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 200/200 [01:48<00:00,  1.84it/s]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 200/200 [02:19<00:00,  1.44it/s]


## ATT-Zero dist

In [4]:
eval_data = attrs[f'sst2_gemma-2b-it-zero']

In [5]:
list(eval_data.columns[5:])

['gradnorm1',
 'gradnorm2',
 'gradinp',
 'erasure',
 'integrated_grad',
 'kernel_shap',
 'random']

In [10]:
lls_list_att_zero = {}
explanation_types = list(eval_data.columns[5:])
percentages = [0.1,0.2,0.3,0.4,0.5]
nums = [0,200]
datasets = ["sst2","news","imdb"]
using_fine_tuned = [False,True]
for ds in datasets:
    for exp in explanation_types:
        for percent in percentages:
            for ft in using_fine_tuned:
                lls_list_att_zero[f"{ds}-{ft}-{exp}-{percent}"] = []

In [11]:
import numpy as np
import torch
methods = ["att-zero"]

pred_models = ["gemma-2b"]


for ds in datasets:

    torch.cuda.empty_cache()
    for pr_model in pred_models:
            
        for ft in using_fine_tuned:
            flipping_dict = defaultdict(list)
            for method in methods:
                

                torch.cuda.empty_cache()

                if ft:
                    pred_model_id = f"predictor_{pr_model}_{ds}_merged"
                    eval_data = attrs[f'{ds}_{pr_model}-ft']
                else:
                    
                    if zero:
                        pred_model_id = f"{pr_model}-it"
                        eval_data = attrs[f'{ds}_{pr_model}-it-zero']
                    else:
                        pred_model_id = f"{pr_model}-it"
                        eval_data = attrs[f'{ds}_{pr_model}-it']


                pred_model = AutoModelForCausalLM.from_pretrained(pred_model_id,device_map=pred_device,
                        torch_dtype=torch.float16,trust_remote_code=True, attn_implementation="flash_attention_2")
                pred_tokenizer = AutoTokenizer.from_pretrained(pred_model_id)
                    # all_tokens = list(pred_tokenizer.get_vocab().keys())
                
                for i in tqdm(range(nums[0],nums[1])):
                    torch.cuda.empty_cache()
                    flipping_dict["sentence"].append(eval_data["comment"].iloc[i])
                    
                    for exp in explanation_types:
                        flipped = False
                        attr = eval_data[exp].iloc[i]


                        for percent in percentages:
                            
                            nlls_org = []
                            num_to_mask = int(np.ceil(percent * len(pred_tokenizer(eval_data.iloc[i].comment)["input_ids"])))
                            idx_to_mask = np.argsort(attr)[-int(num_to_mask):]
                            tokens = pred_tokenizer.convert_ids_to_tokens(pred_tokenizer(eval_data.prompt.iloc[i])["input_ids"])                               
                            sentence = pred_tokenizer.convert_tokens_to_string(tokens)
                            flipping_dict[f"{exp}_masked_sentence_{percent}_predictor_{pr_model}_ft_{ft}_{method}"].append(sentence)
                            
                            predicted_label = eval_data.predicted_id.iloc[i]
                            foil_id = eval_data.foil_id.iloc[i].tolist()
                            foil_sentiment = pred_tokenizer.convert_ids_to_tokens(foil_id)
                            
                            encoding = pred_tokenizer(sentence, return_tensors="pt")
                            
                            ids = encoding.input_ids.to(pred_device)
                            mask = encoding.attention_mask

                            if method == "att-zero":
                                mask[0][idx_to_mask] = 0
                            mask = mask.to(pred_device)
                            labels = ids.clone()
                            out  = pred_model(input_ids = ids, attention_mask = mask,labels=labels)
                            logits = out.logits[0,-1]
                            neg_log_likelihood = out.loss.detach().cpu().numpy()
                            if logits[foil_id] > logits[predicted_label]:
                                if flipped == False:
                                    flipped = True
                                    mask_percent = percent

                            if flipped == False and percent == 0.5:
                                mask_percent = percent
                            nlls_org.append(neg_log_likelihood)
                            lls_list_att_zero[f"{ds}-{ft}-{exp}-{percent}"].append(neg_log_likelihood)

                        flipping_dict[f"{exp}_mask_percent_predictor_{pr_model}_ft_{ft}_{method}"].append(mask_percent)
                        flipping_dict[f"{exp}_succuss_in_flipping_predictor_{pr_model}_ft_{ft}_{method}"].append(flipped)

 
            with open(f"camel_mask2/{ds}_predictor_{pr_model}_ft_{ft}_{nums[0]}_{nums[1]}-att-zero.pcl","+bw") as f:
                pickle.dump(lls_list_att_zero,f)

 


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 200/200 [02:07<00:00,  1.57it/s]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 200/200 [02:05<00:00,  1.59it/s]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

100%|██████████| 200/200 [02:28<00:00,  1.35it/s]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 200/200 [02:41<00:00,  1.24it/s]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 200/200 [02:39<00:00,  1.25it/s]


In [12]:
lls_list_att_zero

{'sst2-False-gradnorm1-0.1': [array(8.380473, dtype=float32),
  array(7.2939134, dtype=float32),
  array(7.8443875, dtype=float32),
  array(7.0930867, dtype=float32),
  array(8.417657, dtype=float32),
  array(8.42703, dtype=float32),
  array(8.266432, dtype=float32),
  array(8.673728, dtype=float32),
  array(7.614702, dtype=float32),
  array(9.251737, dtype=float32),
  array(8.7773905, dtype=float32),
  array(8.091325, dtype=float32),
  array(9.278824, dtype=float32),
  array(7.5965157, dtype=float32),
  array(7.703024, dtype=float32),
  array(7.9593916, dtype=float32),
  array(8.417772, dtype=float32),
  array(7.3942146, dtype=float32),
  array(7.176128, dtype=float32),
  array(6.9626303, dtype=float32),
  array(8.235555, dtype=float32),
  array(7.64813, dtype=float32),
  array(7.0011516, dtype=float32),
  array(7.8061647, dtype=float32),
  array(8.183828, dtype=float32),
  array(7.972577, dtype=float32),
  array(7.168119, dtype=float32),
  array(9.026914, dtype=float32),
  array(8.40